# Calculate the metrics for validate set

In [1]:
import os

niigz_files = os.listdir()
niigz_files.sort()
# niigz_files

groundtruth_files = []
predict_files = []
for niigz_file in niigz_files:
    if "groundtruth" in niigz_file:
        groundtruth_files.append(niigz_file)
    if "predict" in niigz_file:
        predict_files.append(niigz_file)

groundtruth_files.sort()
predict_files.sort()

case_names = []
for file in groundtruth_files:
    case_name = file.split("-groundtruth")[0]
    case_names.append(case_name)

case_names.sort()
print(case_names)

test_dict = dict()
for case_name in case_names:
    test_dict[case_name] = {}
    groundtruth_niigz = case_name + "-groundtruth.nii.gz"
    predict_niigz = case_name + "-predict.nii.gz"
    test_dict[case_name]["groundtruth"] = groundtruth_niigz
    test_dict[case_name]["predict"] = predict_niigz

print(len(test_dict))

['ATM_029_0000', 'ATM_054_0000', 'ATM_055_0000', 'ATM_057_0000', 'ATM_091_0000', 'ATM_174_0000', 'ATM_215_0000', 'ATM_505_0000', 'ATM_688_0000']
9


In [2]:
import SimpleITK as sitk
import numpy as np

def load_CT_scan_3D_image(niigz_file_name):
    itkimage = sitk.ReadImage(niigz_file_name)
    numpyImages = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImages, numpyOrigin, numpySpacing

def false_positive_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fp = np.sum(pred - pred * label) + smooth
    fpr = round(fp * 100 / (np.sum((1.0 - label)) + smooth), 3)
    return fpr

def false_negative_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fn = np.sum(label - pred * label) + smooth
    fnr = round(fn * 100 / (np.sum(label) + smooth), 3)
    return fnr

def sensitivity_calculation(pred, label):   #  identical to True-Positive-Rate
    sensitivity = round(100 - false_negative_rate_calculation(pred, label), 3)
    return sensitivity

def dice_coefficient_score_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    intersection = np.sum(pred * label)
    dice_coefficient_score = round(((2.0 * intersection + smooth) / (np.sum(pred) + np.sum(label) + smooth)) * 100, 2)
    return dice_coefficient_score

def precision_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    tp = np.sum(pred * label) + smooth
    precision = round(tp * 100 / (np.sum(pred) + smooth), 3)
    return precision


In [3]:
for item in test_dict.items():
    case_name = item[0]
    gt_npy, origin, spacing = load_CT_scan_3D_image(item[1]["groundtruth"])
    pred_npy, _, _ = load_CT_scan_3D_image(item[1]["predict"])
    
    FPR = false_positive_rate_calculation(pred_npy, gt_npy)
    FNR = false_negative_rate_calculation(pred_npy, gt_npy)
    Sensitivity = sensitivity_calculation(pred_npy, gt_npy)
    Precision = precision_calculation(pred_npy, gt_npy)
    DSC = dice_coefficient_score_calculation(pred_npy, gt_npy)
    
    print("{0} & {1} & {2} & {3} & {4} & {5}, spacing = {6}"
          .format(case_name, FPR, FNR, Sensitivity, Precision, DSC, spacing))
    

ATM_029_0000 & 0.017 & 9.222 & 90.778 & 93.232 & 91.99, spacing = [0.5     0.78125 0.78125]
ATM_054_0000 & 0.032 & 3.876 & 96.124 & 93.343 & 94.71, spacing = [0.5        0.83007812 0.83007812]
ATM_055_0000 & 0.031 & 3.571 & 96.429 & 91.66 & 93.98, spacing = [0.5       0.8203125 0.8203125]
ATM_057_0000 & 0.03 & 5.757 & 94.243 & 92.35 & 93.29, spacing = [0.5        0.83789062 0.83789062]
ATM_091_0000 & 0.034 & 3.432 & 96.568 & 91.148 & 93.78, spacing = [0.5     0.78125 0.78125]
ATM_174_0000 & 0.019 & 26.864 & 73.136 & 94.026 & 82.28, spacing = [1.79999995 0.625      0.625     ]
ATM_215_0000 & 0.023 & 9.419 & 90.581 & 91.951 & 91.26, spacing = [0.625 0.625 0.625]
ATM_505_0000 & 0.05 & 55.899 & 44.101 & 84.808 & 58.03, spacing = [0.80000001 0.6171875  0.6171875 ]
ATM_688_0000 & 0.023 & 2.217 & 97.783 & 93.409 & 95.55, spacing = [0.5     0.78125 0.78125]
